# Standardizing Event Data for DeRisk

DeRisk currently works with event data in a raw format where relevant data (like the user, token, amount, etc.) is saved in a dictionary or list-like structure in one of the columns. To better extract information from the database, it is helpful to have a unified data structure where all relevant pieces of information are saved in separate columns. This allows for easy querying of all events of a given type (deposit, withdrawal, liquidation) for a given user, lending protocol, et

The following steps outline the process to take a sample of events and convert them to a standardized format that can be used to store information about any type of event and any ing protocolv')


In [1]:
#import libraries
import pandas as pd 
import gcsfs

1.  Load event data from Paraquet file

In [2]:
# Dictionary of Parquet URLs
parquet_urls = {
    "zklend": "https://storage.googleapis.com/derisk-persistent-state/zklend_data/events_sample.parquet",
    "nostra_alpha": "https://storage.googleapis.com/derisk-persistent-state/nostra_alpha_data/events_sample.parquet",
    "nostra_mainnet": "https://storage.googleapis.com/derisk-persistent-state/nostra_mainnet_data/events_sample.parquet",
    "hashstack_v0": "https://storage.googleapis.com/derisk-persistent-state/hashstack_v0_data/events_sample.parquet",
    "hashstack_v1": "https://storage.googleapis.com/derisk-persistent-state/hashstack_v1_data/events_sample.parquet",
}

# Load data from GCS
gcs_df_list = []
for protocol, url in parquet_urls.items():
    fs = gcsfs.GCSFileSystem()
    gcs_path = url.replace('https://storage.googleapis.com/', '')
    with fs.open(gcs_path, 'rb') as f:
        df = pd.read_parquet(f, engine='pyarrow')
        df['Protocol'] = protocol
        gcs_df_list.append(df)

# Combine all GCS DataFrames into one
df_events = pd.concat(gcs_df_list, ignore_index=True)

In [3]:
df_events.head()

,block_hash,block_number,transaction_hash,event_index,from_address,keys,data,timestamp,key_name,Protocol,order
0,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb285127...,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150...,12,0x04c0a5193d58f74fbace4b74dcf65481e734ed171412...,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e...","b'[""0x3fe2b97c1fd336e750087d68b9b867997fd64a26...",1683226849,AccumulatorsSync,zklend,NaN
1,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb285127...,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150...,15,0x04c0a5193d58f74fbace4b74dcf65481e734ed171412...,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e...","b'[""0x68f5c6a61780768455de69077e07e89787839bf8...",1683226849,AccumulatorsSync,zklend,NaN
2,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb285127...,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150...,18,0x04c0a5193d58f74fbace4b74dcf65481e734ed171412...,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e...","b'[""0xda114221cb83fa859dbdb4c44beeaa0bb37c7537...",1683226849,AccumulatorsSync,zklend,NaN
3,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb285127...,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150...,6,0x04c0a5193d58f74fbace4b74dcf65481e734ed171412...,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e...","b'[""0x49d36570d4e46f48e99674bd3fcc84644ddd6b96...",1683226849,AccumulatorsSync,zklend,NaN
4,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb285127...,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150...,9,0x04c0a5193d58f74fbace4b74dcf65481e734ed171412...,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e...","b'[""0x53c91253bc9682c04929ca02ed00b3e423f6710d...",1683226849,AccumulatorsSync,zklend,NaN


In [4]:
# taking a look at the entire second row
pd.set_option('display.max_colwidth', None)
df.iloc[1,:]

block_hash                                                                                                                                                                                                                                                                                                                                                                  0x06c263286120af7afb020f535cd53e17da319df8435058d76450bdcd3ebaecb3
block_number                                                                                                                                                                                                                                                                                                                                                                                                                            273300
transaction_hash                                                                                                                          

2. Define a Function to Decode Byte Strings
Define a helper function to decode byte strings if necessary. This function will be used to ensure that data in the keys and data columns is correctly decoded before further processing.

In [5]:
def decode_byte_string(value):
    return value.decode("utf-8")  

3. Define a Function to Transform Each Row
Define a function that decodes the keys and data columns, converts the hexadecimal strings to integers, and structures the information into a standardized format.

### For zklend Protocol, these are the events and their relevant data.
- for accumulators_sync event: token is in data[0], lending_accumulator is in data[1], debt_accumulator is in data[2]
- for deposit event: user is in data[0], token is in data[1], face_amount is in data[2]
- for collateral_enabled event: user is in data[0], token is in data[1]
- for collateral_disabled event: user is in data[0], token is in data[1]
- for withdrawal event: user is in data[0], token is in data[1], face_amount is in data[2]
- for borrowing event: user is in data[0], token is in data[1], raw_amount is in data[2], face_amount is in data[3]
- for repayment event: user is in data[1], token is in data[2], raw_amount is in data[3]
- for liquidation event: user is in data[1], debt_token is in data[2], debt_raw_amnt is in data[3], collateral_token is in data[5], collateral_face_amnt is in data[6]

In [6]:
# Define a function to decode bytes, convert to list, and extract specific elements based on the event
def extract_zklend(row):
    
    def hex_to_int(hex_str):
        return int(hex_str, 16)
    
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'Deposit':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'face_amount': hex_to_int(data_list[2])
            })
        elif row['key_name'] == 'Withdrawal':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'face_amount': hex_to_int(data_list[2])
            })
        elif row['key_name'] == 'Liquidation':
            return pd.Series({
                'user': data_list[1],
                'debt_token': data_list[2],
                'debt_raw_amount': hex_to_int(data_list[3]),
                'collateral_token': data_list[5],
                'collateral_face_amount': data_list[6]
            })
        elif row['key_name'] == 'CollateralEnabled':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1]
            })
        elif row['key_name'] == 'CollateralDisabled':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1]
            })
        elif row['key_name'] == 'Borrowing':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'raw_amount': hex_to_int(data_list[2]),
                'face_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'Repayment':
            return pd.Series({
                'user': data_list[1],
                'token': data_list[2],
                'raw_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'AccumulatorsSync':
            return pd.Series({
                'token': data_list[0],
                'lending_accumulator': data_list[1],
                'debt_accumulator': data_list[2]
            })
        else:
            return pd.Series()
    except Exception as e:
        # Handle any parsing errors
        return pd.Series()

# Filter the dataset for 'zklend' protocol
zk_data = df_events[df_events['Protocol'] == 'zklend']

# Apply the extraction function to each row
extracted_elements = zk_data.apply(extract_zklend, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_zk = pd.concat([zk_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# display the dataframe
# standard_zk.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_standard_zk_events.csv'
# standard_zk.to_csv(output_file_path, index=False)

### For nostra_alpha protocol, these are the events and their relevant data
- for debt event: user is in data[0], token is in from_address, face_amount is in data[1]
- for mint event: user is in data[0], token is in from_address, face_amount is in [1]
- for interest state updated event :  debtToken is in data[0], lendingRate is in data[1], borrowRate is in data[3], lendIndex is in [5], borrowIndex is in [7]

In [7]:
# Define a function to decode bytes, convert to list, and extract specific elements based on the event
def extract_nostra_alpha(row):
    
    def hex_to_int(hex_str):
        return int(hex_str, 16)
    
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
   
        if row['key_name'] == 'Mint':
            return pd.Series({
                'user': data_list[0],
                'token': row['from_address'],
                'face_amount': hex_to_int(data_list[1])
            })
        elif row['key_name'] == 'Debt':
            return pd.Series({
                'user': data_list[0],
                'token': row['from_address'],
                'face_amount': hex_to_int(data_list[1])
            })
        elif row['key_name'] == 'InterestStateUpdated':
            return pd.Series({
                'debt_token': data_list[0],
                'lending_rate': data_list[1],
                'borrow_rate': data_list[3],
                'lend_index': data_list[5],
                'borrow_index': data_list[7]
            })
        else:
            return pd.Series()  # Return an empty series for unmatched events
    except Exception as e:
        # Handle any parsing errors
        return pd.Series()  # Return an empty series in case of error

# Filter the dataset for 'nostra_alpha' protocol
nostra_alpha_data = df_events[df_events['Protocol'] == 'nostra_alpha']

# Apply the extraction function to each row
extracted_elements = nostra_alpha_data.apply(extract_nostra_alpha, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_nalpha = pd.concat([nostra_alpha_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_standard_nalpha_events.csv'
# standard_nalpha.to_csv(output_file_path, index=False)

### For hashstack_v0 protocol these are the events and their relevant data:
- for new loan: loan_id is in data[0]int, user is in data[1], debt_token is in data[2], debt_face_amount is in data[4]int,
                borrowed_collateral_token is in data[6],borrowed_collateral_face_amount is in data[7]int, debt_category is in data[10]int
- for loan_withdrawal: loan_id is in data[0]int, user is in data[1], debt_token is in data[2], debt_face_amount is in data[4]int,
                borrowed_collateral_token is in data[6],borrowed_collateral_face_amount is in data[7]int, debt_category is in data[10]int
- for loan_interest_deducted: load_id is in data[11], original_collateral_token is in data[0], original_collateral_face_amount is in data[3]int
- for loan_swap: old_loan_id is in data[0]int, old_user is in data[1], new_loan_id is in data[14]int, new_user is in data[15], new_debt_token is in data[16], new_debt_face_amount is in data[18]int,  new_borrowed_collateral_token is in data[20], new_borrowed_collateral_face_amount is in data[21]int, new_debt_category is in data[24]
- for collateral_added: loan_id is in data[9], original_collateral_token is in data[0], original_collateral_face_amount is in data[3]int
- for loan_repaid: loan_id is on data[0], user is in data[1], debt_token is in data[2], borrowed_collateral_token is in data[6], borrowed_colleteral_face_amount in in data[7], debt_category is in data[10]
- for collateral_withdrawal: loan_id is in data[9], original_collateral_token is in data[0], original_collateral_face_amount is in data[3]int,
- for liquidated_event: loan_id is in data[0]int, user is in data[1], debt_token is in data[2], borrowed_collateral_token is in data[6], borrowed_collateral_face_amount is in data[7]int, debt_category is in data[10]int
        

In [8]:
def extract_hashstack_v0(row):
    
    def hex_to_int(hex_str):
        return int(hex_str, 16)
    
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'new loan' or row['key_name'] == 'loan_withdrawal':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'user': data_list[1],
                'debt_token': data_list[2],
                'debt_face_amount': hex_to_int(data_list[4]),
                'borrowed_collateral_token': data_list[6],
                'borrowed_collateral_face_amount': hex_to_int(data_list[7]),
                'debt_category': hex_to_int(data_list[10])
            })
        elif row['key_name'] == 'loan_interest_deducted':
            return pd.Series({
                'loan_id': data_list[11],
                'original_collateral_token': data_list[0],
                'original_collateral_face_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'loan_swap':
            return pd.Series({
                'old_loan_id': hex_to_int(data_list[0]),
                'old_user': data_list[1],
                'new_loan_id': hex_to_int(data_list[14]),
                'new_user': data_list[15],
                'new_debt_token': data_list[16],
                'new_debt_face_amount': hex_to_int(data_list[18]),
                'new_borrowed_collateral_token': data_list[20],
                'new_borrowed_collateral_face_amount': hex_to_int(data_list[21]),
                'new_debt_category': data_list[24]
            })
        elif row['key_name'] == 'collateral_added':
            return pd.Series({
                'loan_id': data_list[9],
                'original_collateral_token': data_list[0],
                'original_collateral_face_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'loan_repaid':
            return pd.Series({
                'loan_id': data_list[0],
                'user': data_list[1],
                'debt_token': data_list[2],
                'borrowed_collateral_token': data_list[6],
                'borrowed_collateral_face_amount': data_list[7],
                'debt_category': data_list[10]
            })
        elif row['key_name'] == 'collateral_withdrawal':
            return pd.Series({
                'loan_id': data_list[9],
                'original_collateral_token': data_list[0],
                'original_collateral_face_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'liquidated_event':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'user': data_list[1],
                'debt_token': data_list[2],
                'borrowed_collateral_token': data_list[6],
                'borrowed_collateral_face_amount': hex_to_int(data_list[7]),
                'debt_category': hex_to_int(data_list[10])
            })
        else:
            return pd.Series()
    except Exception as e:
        # Handle any parsing errors
        return pd.Series()

# Filter the dataset for 'hashstack_v0' protocol
hashstack_v0_data = df_events[df_events['Protocol'] == 'hashstack_v0']

# Apply the extraction function to each row
extracted_elements = hashstack_v0_data.apply(extract_hashstack_v0, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_hashstack_v0 = pd.concat([hashstack_v0_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# Display the resulting dataframe
# standard_hashstack_v0.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_hashstack_v0_events.csv'
# standard_hashstack_v0.to_csv(output_file_path, index=False)

### for hashstack_v1 protocol, these are the following events and their relevant data:
- for new_loan event: loan_id is in data[0]int, user is in data[1], collateral_loan_id is in data[12]int, debt_token is in data[2], debt_face_amount is in data[3]int, borrowed_collateral_token is in data[5], borrowed_collateral_face_amount is in data[6], original_collateral_token is in data[13], original_collateral_face_amount is in data[14]int
- for loan_spent: old_loan_id is in data[0]int, old_user is iin data[1], new_loan_id isin data[12], new_user isin data[13], new_debt_token isin data[14], new_debt_face_amount isin data[15]int,  new_borrowed_collateral_token isin data[17], new_borrowed_collateral_face_amount isin data[18]
- for collateral_added: loan_id is in data[0]int,  original_collateral_token is in data[1], original_collateral_face_amount is in data[2]
- for load_repaid: old_loan_id is in data[0], old_user is in data[1], new_loan_id is in data[12]int, new_user is in data[13],
new_collateral_loan_id is in data[24]int, new_debt_token is in data[14],new_debt_face_amount is in data[15]int, new_borrowed_collateral_token is in data[17], new_borrowed_collateral_face_amount is in data[18]int, new_original_collateral_token is in data[25], new_original_collateral_face_amount is in data[26]
- for loan_transfered: loan_id is in data[0]int, old_user is in data[1], new_user is in data[2]

In [9]:
# Define a function to decode bytes, convert to list, and extract specific elements based on the event
def extract_hashstack_v1(row):
    
    def hex_to_int(hex_str):
        return int(hex_str, 16)
    
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'new_loan':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'user': data_list[1],
                'collateral_loan_id': hex_to_int(data_list[12]),
                'debt_token': data_list[2],
                'debt_face_amount': hex_to_int(data_list[3]),
                'borrowed_collateral_token': data_list[5],
                'borrowed_collateral_face_amount': hex_to_int(data_list[6]),
                'original_collateral_token': data_list[13],
                'original_collateral_face_amount': hex_to_int(data_list[14])
            })
        elif row['key_name'] == 'loan_spent':
            return pd.Series({
                'old_loan_id': hex_to_int(data_list[0]),
                'old_user': data_list[1],
                'new_loan_id': data_list[12],
                'new_user': data_list[13],
                'new_debt_token': data_list[14],
                'new_debt_face_amount': hex_to_int(data_list[15]),
                'new_borrowed_collateral_token': data_list[17],
                'new_borrowed_collateral_face_amount': data_list[18]
            })
        elif row['key_name'] == 'collateral_added':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'original_collateral_token': data_list[1],
                'original_collateral_face_amount': hex_to_int(data_list[2])
            })
        elif row['key_name'] == 'loan_repaid':
            return pd.Series({
                'old_loan_id': data_list[0],
                'old_user': data_list[1],
                'new_loan_id': hex_to_int(data_list[12]),
                'new_user': data_list[13],
                'new_collateral_loan_id': hex_to_int(data_list[24]),
                'new_debt_token': data_list[14],
                'new_debt_face_amount': hex_to_int(data_list[15]),
                'new_borrowed_collateral_token': data_list[17],
                'new_borrowed_collateral_face_amount': hex_to_int(data_list[18]),
                'new_original_collateral_token': data_list[25],
                'new_original_collateral_face_amount': hex_to_int(data_list[26])
            })
        elif row['key_name'] == 'loan_transfered':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'old_user': data_list[1],
                'new_user': data_list[2]
            })
        else:
            return pd.Series()
    except Exception as e:
        # Handle any parsing errors
        return pd.Series()

# Filter the dataset for 'hashstack_v1' protocol
hashstack_v1_data = df_events[df_events['Protocol'] == 'hashstack_v1']

# Apply the extraction function to each row
extracted_elements = hashstack_v1_data.apply(extract_hashstack_v1, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_hashstack_v1 = pd.concat([hashstack_v1_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# Display the resulting dataframe
standard_hashstack_v1.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_hashstack_v1_events.csv'
# standard_hashstack_v1.to_csv(output_file_path, index=False)

,block_hash,block_number,transaction_hash,timestamp,key_name,borrowed_collateral_face_amount,borrowed_collateral_token,collateral_loan_id,debt_face_amount,debt_token,...,new_debt_token,new_loan_id,new_original_collateral_face_amount,new_original_collateral_token,new_user,old_loan_id,old_user,original_collateral_face_amount,original_collateral_token,user
300780,0x068b04151068a7acc1f72c9b2023e176037cb6bce171a5e3abaf6fb961f4e1a2,273262,0x0676e3ae850f6fae3da1269b9c390f74c26eb1948ca2bd1e16a47fa9399ca84d,1695977183,new_loan,4500000,0x68f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8,1.0,4500000,0x12b8185e237dd0340340faeb3351dbe53f8a42f5a9bf974ddf90ced56e301c7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000000,0x5fa6cc6185eab4b0264a4134e2d4e74be11205351c7c91196cb27d5d97f8d21,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a
300781,0x06c263286120af7afb020f535cd53e17da319df8435058d76450bdcd3ebaecb3,273300,0x06ebb5723abf3878214b66bd4bbe7087280707da7231b074a1bcfba8b5ec78d7,1695977763,new_loan,2400000000000000,0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7,2.0,2400000000000000,0x1ef7f9f8bf01678dc6d27e2c26fb7e8eac3812a24752e6a1d6a49d153bec9f3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000000,0x5fa6cc6185eab4b0264a4134e2d4e74be11205351c7c91196cb27d5d97f8d21,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a
300782,0x07d0f1faf834b7906e4de54c440d0aa35112bc1a243c12e63e4f8b6006e857df,273372,0x0051f75ef1e08f70d1c8efe7866384d026aa0ca092ded8bd1c903aac0478b990,1695978978,loan_spent,NaN,NaN,NaN,NaN,NaN,...,0x12b8185e237dd0340340faeb3351dbe53f8a42f5a9bf974ddf90ced56e301c7,0x1,NaN,NaN,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,1,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,NaN,NaN,NaN
300783,0x06f650deb66bb602f02bd87f29aa7c28909b126f9d7c738f8a3ad200baa888c4,273387,0x01bf0af4c1922e2b8cc523e6e088d72885701e9416058a3e6bcc2bcf7f0bd5d0,1695979201,collateral_added,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3999998,0x5fa6cc6185eab4b0264a4134e2d4e74be11205351c7c91196cb27d5d97f8d21,NaN
300784,0x06384cf4b11477d048623b43e45970c5a4a10653706547b76e92096f7d421774,273396,0x0389888c9ef7dfe1491a3d15afa79b565bc52dccb01d512693f786f6c61fdc0f,1695979342,collateral_added,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4999998,0x5fa6cc6185eab4b0264a4134e2d4e74be11205351c7c91196cb27d5d97f8d21,NaN


### for nostra_mainnet, the same extraction will be applied as with nostra_alpha.

In [10]:
# Filter the dataset for 'nostra_alpha' protocol
nostra_mainnet_data = df_events[df_events['Protocol'] == 'nostra_mainnet']

# Apply the extraction function to each row
extracted_elements = nostra_mainnet_data.apply(extract_nostra_alpha, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_mainnet = pd.concat([nostra_mainnet_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# Display the resulting dataframe
standard_mainnet.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_standard_mainnet_events.csv'
# standard_mainnet.to_csv(output_file_path, index=False)

,block_hash,block_number,transaction_hash,timestamp,key_name,borrow_index,borrow_rate,debt_token,face_amount,lend_index,lending_rate,token,user
200000,0x051eddd5b8502044947dc35d687caa2b2634d3dbb670d3f1f28654fffdcdc892,169098,0x00489af46e28392d1c3e4007476328ba4ccf4bd84f4f5565fda0888d5518a70b,1692858419,Mint,NaN,NaN,NaN,1000000000000000000,NaN,NaN,0x04f18ffc850cdfa223a530d7246d3c6fc12a5969e0aa5d4a88f470f5fe6c46e9,0x5295882956890bb9c05e54c1e7efa903a4a21083ebd6a3c175507f74e549574
200001,0x051eddd5b8502044947dc35d687caa2b2634d3dbb670d3f1f28654fffdcdc892,169098,0x00489af46e28392d1c3e4007476328ba4ccf4bd84f4f5565fda0888d5518a70b,1692858419,InterestStateUpdated,0xde0b6b3a7640000,0x0,0x66037c083c33330a8460a65e4748ceec275bbf5f28aa71b686cbc0010e12597,NaN,0xde0b6b3a7640000,0x0,NaN,NaN
200002,0x0068928198ae9c350e91761752850081262aede8cabd5824341c905b5750fae5,169911,0x01bf3b09632626f8cc8ed46a1b394df8ddf2776b5f04794ad740e335b1ec5b9a,1692881265,Mint,NaN,NaN,NaN,100000,NaN,NaN,0x05dcd26c25d9d8fd9fc860038dcb6e4d835e524eb8a85213a8cda5b7fff845f6,0x2d50b82f53cc24f44d562207009f89f60b04ba98bff6deba999b50b964c9dfe
200003,0x0068928198ae9c350e91761752850081262aede8cabd5824341c905b5750fae5,169911,0x01bf3b09632626f8cc8ed46a1b394df8ddf2776b5f04794ad740e335b1ec5b9a,1692881265,InterestStateUpdated,0xde0b6b3a7640000,0x0,0x63d69ae657bd2f40337c39bf35a870ac27ddf91e6623c2f52529db4c1619a51,NaN,0xde0b6b3a7640000,0x0,NaN,NaN
200004,0x02ce054477550a921fcd804daf2482d9503562dfb4790568aba6783184dd5656,175894,0x003e1ef7eca78dff0bf0e181658360ecf2a5f0d0380b5b9c1f239e3f2b057809,1693155334,Mint,NaN,NaN,NaN,25007406626554445,NaN,NaN,0x057146f6409deb4c9fa12866915dd952aa07c1eb2752e451d7f3b042086bdeb8,0x1b706b18846667c44fdf2dc302ed90b98c0ae3aa6e041fdbdea1289d0284b03
